### Yield From - Sending Data

yield from establishes a 2-way communication channel

- between caller and subgenerator

We know that before we can send() to a coroutine, we have to prime it first

How does this work with yield from?

In [1]:
def delegator():
    yield from coro()
    
def coro():
    while True:
        received = yield
        print(received)

In [2]:
d = delegator()

In [3]:
next(d)

What about coro()?

yield from will automatically prime the coroutine when necessary

#### Sending data to the subgenerator

Once the delegator has been primed data can be sent to it using send()

In [4]:
d.send('python')

python


#### Control Flow

meh, you know this by this point

#### Multiple Delegators -> Pipeline

When you chain multiple delegators, you get the pipeline. Refer to code example

#### Code Examples

In [5]:
def echo():
    while True:
        received = yield
        print(received[::-1])

In [6]:
e = echo()
next(e)

In [7]:
e.send('stressed')

desserts


In [8]:
e.send('tons')

snot


In [9]:
e.close()

In [10]:
def delegator():
    e = echo()
    yield from e

In [12]:
d = delegator()
next(d)

In [13]:
from inspect import getgeneratorstate, getgeneratorlocals

In [14]:
getgeneratorlocals(d)

{'e': <generator object echo at 0x0000014A761C4B48>}

In [15]:
e = getgeneratorlocals(d)['e']

In [16]:
print(getgeneratorstate(d))
print(getgeneratorstate(e))

GEN_SUSPENDED
GEN_SUSPENDED


In [17]:
d.send('stressed')

desserts


In [18]:
d.send('tons')

snot


In [23]:
def echo():
    output = None
    while True:
        received = yield output
        output = received[::-1]

In [24]:
e = echo()
result = next(e)
print(type(result))

<class 'NoneType'>


In [26]:
result = e.send('stressed')

In [27]:
result

'desserts'

In [28]:
def delegator():
    yield from echo()

In [29]:
d = delegator()

In [30]:
next(d)

In [31]:
result = d.send('stressed')

In [32]:
result

'desserts'

In [33]:
l = [1, 2, [3, 4, [5, 6]], [7, [8, 9, 10]]]

In [34]:
def flatten(curr_item):
    if isinstance(curr_item, list):
        for item in curr_item:
            flatten(item)
    else:
        print(curr_item)

In [35]:
flatten(l)

1
2
3
4
5
6
7
8
9
10


In [36]:
def flatten(curr_item, output):
    if isinstance(curr_item, list):
        for item in curr_item:
            flatten(item, output)
    else:
        output.append(curr_item)

In [37]:
output = []
flatten(l, output)

In [38]:
print(output)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [39]:
def flatten_gen(curr_item):
    if isinstance(curr_item, list):
        for item in curr_item:
            yield from flatten_gen(item)
    else:
        yield curr_item

In [40]:
for item in flatten_gen(l):
    print(item)

1
2
3
4
5
6
7
8
9
10


In [41]:
list(flatten_gen(l))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [42]:
def is_iterable(item):
    try:
        iter(item)
    except:
        return False
    else:
        return True

In [45]:
def flatten_gen(curr_item):
    if is_iterable(curr_item):
        for item in curr_item:
            yield from flatten_gen(item)
    else:
        yield curr_item

In [46]:
list(flatten_gen(l))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [47]:
l = [1, 2, (3, 4, {5, 6}), (7, 8, [9, 10])]

In [48]:
list(flatten_gen(l))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [50]:
l = ['abc', [1, 2, (3, 4)]]

In [52]:
list(flatten_gen(l))

['a', 'b', 'c', 1, 2, 3, 4]

In the video example, Fred gets a "maximum recursion depth exceeded"

This *was* because individual chars are iterable

In [54]:
is_iterable('a')

True

However, I guess this has been corrected in this version

Here is what Fred did to fix this:

In [56]:
def is_iterable(item, *, str_is_iterable=True):
    try:
        iter(item)
    except:
        return False
    else:
        if isinstance(item, str):
            if str_is_iterable and len(item) > 1:
                return True
            else:
                return False
        else:
            return True

In [57]:
print(is_iterable([1, 2, 3]))
print(is_iterable('abc'))
print(is_iterable('a'))

True
True
False


In [59]:
print(is_iterable([1, 2, 3], str_is_iterable=False))
print(is_iterable('abc', str_is_iterable=False))
print(is_iterable('a', str_is_iterable=False))

True
False
False


In [62]:
def flatten_gen(curr_item, *, str_is_iterable=True):
    if is_iterable(curr_item, str_is_iterable=str_is_iterable):
        for item in curr_item:
            yield from flatten_gen(item, str_is_iterable=str_is_iterable)
    else:
        yield curr_item

In [63]:
list(flatten_gen(l))

['a', 'b', 'c', 1, 2, 3, 4]

In [64]:
list(flatten_gen(l, str_is_iterable=False))

['abc', 1, 2, 3, 4]

#### Quick Pipeline Example

In [65]:
def coro():
    while True:
        received = yield
        print(received)

def gen1():
    yield from gen2()
    
def gen2():
    yield from gen3()
    
def gen3():
    yield from coro()

caller <--> gen1 <--> gen2 <--> gen3 <--> coro

In [66]:
g = gen1()

In [67]:
next(g)

Now gen2, gen3, and coro are created and primed!

In [69]:
g.send('hello')

hello


In [70]:
def coro():
    while True:
        received = yield 100
        print(received)

def gen1():
    yield from gen2()
    
def gen2():
    yield from gen3()
    
def gen3():
    yield from coro()

In [71]:
g = gen1()
next(g)
g.send('hello')

hello


100

And so a value was send and received!